## 구글 드라이브 마운트
colab 런타임을 본인의 구글 드라이브에 마운트시킨 후, 적절한 경로를 이동해 줍니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from google.colab import drive

# drive.mount("/content/drive", force_remount=True)

# 경로로 이동해 주세요.
path = '/content/drive/MyDrive/understood (MI1)/bert_classification_kor/eojeol'
os.chdir(path)

In [ ]:
os.listdir()

['bert_to_keras.py',
 'inference.py',
 'eval_bert_finetuning.py',
 'utils.py',
 'train_bert_finetuning.py',
 'prepare_data.py',
 'modeling.py',
 '.DS_Store',
 'readers',
 'models',
 'vectorizers',
 'layers',
 'starbucks_data',
 '004_bert_eojeol_tensorflow',
 '__pycache__',
 'bert-module',
 'data',
 'KorBERT_classification.ipynb',
 'train_log.png',
 'starbucks_model']

## 환경 설정
KorBERT를 활용할 수 있는 환경을 설정해 줍니다. 우선 아래와 같은 모듈들을 설치해 줍니다.
  - tensorflow-gpu==0.15
  - tensorflow-hub==0.7.0


그 후, 사용 가능한 gpu를 환경변수 CUDA_VISIBLE_DEVICES 변수에 설정해 줍니다.

In [ ]:
!pip install tensorflow-gpu==1.15
!pip install tensorflow-hub==0.7.0

     |████████████████████████████████| 411.5MB 36kB/s 
     |████████████████████████████████| 512kB 46.4MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 3.8MB 30.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=4b5af2b6e1af51aa3b247985af6cdf09cd45db648b6a0e224c9a68ab11a8191e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement tensorflow-estimator<2.5.0,>=2.4.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing instal

사용 가능한 gpu를 확인해 줍니다. 만일 런타임 유형 변경에서 가속기로 gpu를 설정하지 않는다면 다음과 같은 에러 메시지가 나올 것입니다.

`NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.`

이럴 경우 런타임 유형을 변경한 후 다시 실행합니다.

In [ ]:
!export CUDA_VISIBLE_DEVICES=0 # 0번 gpu를 쓰겠다
!nvidia-smi 

Tue Apr 20 00:58:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# checkpoint -> keras layer
ETRI의 KorBERT를 keras model로 변환해 줍니다. 이는 `bert_to_keras.py`로 가능합니다.

In [ ]:
# bert_module로 변환해 줬으면 이 과정은 생략하셔도 됩니다.
# kobert -> 케라스 레이어로 변환
!python bert_to_keras.py 

From bert_to_keras.py:29: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

From /content/drive/My Drive/understood (MI1)/bert_classification_kor/eojeol/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

0.02 ---------
From /content/drive/My Drive/understood (MI1)/bert_classification_kor/eojeol/modeling.py:194: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

From /content/drive/My Drive/understood (MI1)/bert_classification_kor/eojeol/modeling.py:435: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

From /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to la

In [ ]:
# 'bert-module'이라는 경로가 추가적으로 생성된 것을 확인할 수 있습니다.
os.listdir()

['bert_to_keras.py',
 'inference.py',
 'eval_bert_finetuning.py',
 'utils.py',
 'train_bert_finetuning.py',
 'prepare_data.py',
 'modeling.py',
 '.DS_Store',
 'readers',
 'models',
 'vectorizers',
 'layers',
 'starbucks_data',
 '004_bert_eojeol_tensorflow',
 '__pycache__',
 'bert-module',
 'data',
 'KorBERT_classification.ipynb',
 'train_log.png',
 'starbucks_model']

## 데이터 전처리
KorBERT가 Fine-Tune을 할 수 있게끔 데이터를 준비해 주어야 합니다. 준비해야 하는 데이터 파일은 tab(\t)을 기준으로 label과 input sequence가 나뉘어 있어야 합니다.

`prepare_data.py`를 실행할 경우 data -> label, seq.in 으로 나뉘게 되는데, 이떄 seq.in은 KorBERT가 pre-train된 대로 tokenization 과정을 거치게 됩니다.

`prepare_data.py`의 경우 mandatory한 인자로 --input(-i)와 --output(-o)를 갖습니다.

`python prepare_data.py --input ${input_data} --output ${processed_data_path}`


In [ ]:
!pwd

/content/drive/My Drive/understood (MI1)/bert_classification_kor/eojeol


In [ ]:
!python prepare_data.py --input ./starbucks_data/train/train_data1.txt --output starbucks_data/train/

KorBERT의 tokenization 방식을 알기 위해서는 다음 코드와

In [ ]:
# from vectorizers import tokenizationK as tk


# sample_sentence = '와 이 커피 진짜 맛있다.'
# ### 소스코드를 참조하여 알맞게 채워보세요 ###
# vocab_file_path = ''

# tokenizer = tk.FullTokenizer(vocab_file_path)

# tokens = ""

# for word in sample_sentence.split():
#     word_tokens = " ".join(tokenizer.tokenize(word))
#     tokens += word_tokens + " "

## Fine-Tuning 훈련

준비된 데이터를 바탕으로 KorBERT fine-tuning을 실시해 줍니다.

`train_bert_finetuning.py`를 실행하며, mandatory 인자로는 --train(-t) 와 --save(-s)를 받습니다.

`python train_bert_finetuning.py --train ${processed_data_path} --save ${saved_model}`

In [ ]:
!python train_bert_finetuning.py --train starbucks_data/train/ --save starbucks_model


2021-04-20 00:59:48.253922: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-04-20 00:59:48.259568: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-04-20 00:59:48.260155: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56061924c840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-04-20 00:59:48.260195: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-04-20 00:59:48.262253: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-20 00:59:48.375723: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-20 00:59:48.376796:

In [ ]:
!python prepare_data.py --input ./starbucks_data/test/test.txt --output starbucks_data/test/

In [ ]:
!python eval_bert_finetuning.py --model starbucks_model --data starbucks_data/test/



2021-04-20 03:00:01.012766: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2021-04-20 03:00:01.018223: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-04-20 03:00:01.018711: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x561f93560bc0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-04-20 03:00:01.018746: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-04-20 03:00:01.020675: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-04-20 03:00:01.024471: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-04-20 03:00:01.024525: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:16